# NI USB-6002 Data Acquisition (GEEN 3853)

This notebook:
- Detects connected NI DAQ devices
- Acquires **one analog voltage channel** (`ai0`) from an NI USB-6002 (or similar)
- Plots the **time-domain** signal
- Exports acquired data to a **CSV** file (Excel-friendly)

## Prerequisites
1. **NI-DAQmx driver** installed (and device visible in NI MAX). Note: Will only work on Windows Machines. 
2. Python package `nidaqmx` installed in this environment.

It is highly recommended you run this script on an ITLP lab station


## 1) Imports
Run this cell first. If imports fail, confirm your conda environment and that `nidaqmx` is installed.

In [4]:
import time
from datetime import datetime

import numpy as np
import pandas as pd

import nidaqmx
from nidaqmx.constants import AcquisitionType, TerminalConfiguration
from nidaqmx.stream_readers import AnalogSingleChannelReader
from nidaqmx.system import System


ModuleNotFoundError: No module named 'nidaqmx'

## 2) Detect NI DAQ devices
This should list your device (often `Dev1`). If nothing appears, check:
- USB connection
- NI-DAQmx installation
- NI MAX can see the device


In [ ]:
system = System.local()
print('NI-DAQmx driver version:', system.driver_version)

if len(system.devices) == 0:
    print('No NI DAQ devices found. Check NI MAX and your USB connection.')
else:
    for dev in system.devices:
        print('Device:', dev.name, '| Product:', dev.product_type)


## 3) Acquisition settings
Edit these values as needed.

**Sampling**:
- `SAMPLE_RATE_HZ` The **rate** at which you'd like to sample
- `DURATION_S` How long to run samples for

**Input Range**:
- By Default this is set to -10.0 to 10.0 V. The range is divided by the ADC (2^16 bins) so the resolution is 0.31 mV. If you'd like higher resoluiton, you can change the input range (e.g., if you're reading a photo cell with 5V excitation, you might set the input range to 0 to 5 V so your resolution is 5V/2^16=0.076 mV
    
**Terminal configuration** depends on wiring:
- `RSE` (Referenced Single-Ended) is common for quick single-ended measurements referenced to AI GND. This is analogous to `analogRead` on Arduino
- `DIFFERENTIAL` can reduce noise if wired differentially.


In [9]:
# ---- Hardware channel ----
DEVICE = 'Dev1'          # <-- change if needed (from the detection cell)
CHANNEL = 'ai0'          # analog input channel 0
PHYS_CHAN = f"{DEVICE}/{CHANNEL}"

# ---- Sampling ----
SAMPLE_RATE_HZ = 1000.0  # Hz (Would not recommend more than ~10,000 Hz
DURATION_S = 10.0        # seconds to record
CHUNK_SAMPLES = 200      # buffer, saves computer overhead

# ---- Input range (volts) ----
MIN_V = -10.0
MAX_V = 10.0

# ---- Terminal configuration ----
TERM_CFG = TerminalConfiguration.RSE
# TERM_CFG = TerminalConfiguration.DIFFERENTIAL


NameError: name 'TerminalConfiguration' is not defined

## 4) Acquire data from `ai0`
This cell configures a DAQmx Task with a hardware sample clock and reads data in chunks.
At the end, you will have:
- `t` : time vector (seconds)
- `data` : measured voltages (volts)


In [ ]:
num_total_samples = int(SAMPLE_RATE_HZ * DURATION_S)
num_chunks = int(np.ceil(num_total_samples / CHUNK_SAMPLES))

data = np.empty(num_total_samples, dtype=np.float64)
t = np.empty(num_total_samples, dtype=np.float64)

start_wall = datetime.now()

with nidaqmx.Task() as task:
    # Add an analog input voltage channel
    task.ai_channels.add_ai_voltage_chan(
        PHYS_CHAN,
        min_val=MIN_V,
        max_val=MAX_V,
        terminal_config=TERM_CFG,
    )

    # Configure hardware timing
    task.timing.cfg_samp_clk_timing(
        rate=SAMPLE_RATE_HZ,
        sample_mode=AcquisitionType.CONTINUOUS,
        samps_per_chan=CHUNK_SAMPLES,
    )

    # Efficient stream reader
    reader = AnalogSingleChannelReader(task.in_stream)

    task.start()
    idx = 0

    for _ in range(num_chunks):
        n = min(CHUNK_SAMPLES, num_total_samples - idx)
        buf = np.empty(n, dtype=np.float64)

        # Read n samples (blocks until samples are available)
        reader.read_many_sample(
            data=buf,
            number_of_samples_per_channel=n,
            timeout=10.0
        )

        data[idx:idx+n] = buf
        t[idx:idx+n] = (np.arange(idx, idx+n) / SAMPLE_RATE_HZ)
        idx += n

    task.stop()

print(f"Acquired {len(data)} samples from {PHYS_CHAN} @ {SAMPLE_RATE_HZ} Hz")


## 5) Plot time-domain signal
A quick check: if your wiring is correct, the signal should look reasonable.


In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.plot(t, data)
plt.xlabel('Time (s)')
plt.ylabel('Voltage (V)')
plt.title(f'{PHYS_CHAN}  |  {SAMPLE_RATE_HZ} Hz  |  {DURATION_S} s')
plt.grid(True)
plt.show()


## 6) Export CSV for Excel
Exports two columns:
- `time_s`
- `voltage_V`


In [ ]:
df = pd.DataFrame({
    'time_s': t,
    'voltage_V': data
})

fname = f"usb6002_ai0_{start_wall.strftime('%Y%m%d_%H%M%S')}.csv"
df.to_csv(fname, index=False)
print('Wrote:', fname)
df.head()


## 6) Fourier analysis (FFT) Optional)
This converts the time-domain signal into the frequency domain.

**Key ideas**:
- The maximum meaningful frequency is the **Nyquist frequency**: `fs/2`
- Frequency resolution is `Δf = fs/N` (record longer for finer resolution)
- Windowing (Hann) reduces spectral leakage for non-integer cycle captures


In [ ]:
# ---------------------------------------------
# Fourier Analysis of NI-DAQ Data
# ---------------------------------------------
x = data.copy()
fs = float(SAMPLE_RATE_HZ)
N = len(x)

# 1) Remove DC offset
x = x - np.mean(x)

# 2) Apply window to reduce spectral leakage
window = np.hanning(N)
x_windowed = x * window

# 3) Compute single-sided FFT
X = np.fft.rfft(x_windowed)
freq = np.fft.rfftfreq(N, d=1/fs)

# 4) Scale to approximate amplitude in Volts
coherent_gain = np.mean(window)
amplitude = (2.0 / (N * coherent_gain)) * np.abs(X)
amplitude[0] = amplitude[0] / 2.0  # don't double-count DC

# 5) Plot spectrum
f_max = min(200.0, fs / 2)
mask = freq <= f_max

plt.figure()
plt.plot(freq[mask], amplitude[mask])
plt.xlabel('Frequency (Hz)')
plt.ylabel('Amplitude (V)')
plt.title('Single-Sided Amplitude Spectrum')
plt.grid(True)
plt.show()
